In [ ]:
import torch as t
from tqdm import tqdm
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
from torchvision import transforms
import torch.nn.functional as F
import torchvision.transforms.functional as TF

import matplotlib.pyplot as plt
from PIL import Image
import os
import random

from dataset import seg_dataset
from unet_a import unet

C:\Users\AI-CAMERA\anaconda3\envs\pytorch_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
EPOCHS = 50
DATA_PATH = "F:\\CBIR\\data\\combined"
MODEL_SAVE_PATH = "F:\\CBIR\\UNET\\models"
DEVICE = 'cuda' if t.cuda.is_available() else 'cpu'

data = seg_dataset(DATA_PATH)
generator = t.Generator().manual_seed(42)

train_dataset, val_dataset = random_split(dataset=data, lengths=(0.8, 0.2), generator=generator)

train_dataloader = DataLoader(train_dataset, BATCH_SIZE, True)
val_dataloader = DataLoader(val_dataset, BATCH_SIZE, True)

model = unet(in_channel=3, num_classes=1).to(DEVICE)
# model_path = 'D:\\New method template removal\\SWIN TRAINING\\IMAGE ENHANCEMENT\\Fine tuning for kyoshin\\UNET\\models\\Image_enhancement_sd-base.pt'
# model.load_state_dict(t.load(model_path, map_location=t.device(DEVICE), weights_only=True))
optimizer = optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
loss = nn.BCEWithLogitsLoss()

single_img = "F:\\CBIR\\data\\combined\\X\\16353.jpeg"
single_target = "F:\\CBIR\\data\\combined\\Y\\16353.png"
img = Image.open(single_img).convert('RGB')
msk = Image.open(single_target).convert('L')

# transforms_pipe = transforms.Compose([
#         transforms.Resize((512,512)),
#         transforms.ToTensor()
#     ])

angles = [0, 90, 180, 270]
transforms_pipe = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

img_transformed = transforms_pipe(img)
msk_transformed = transforms_pipe(msk)

In [3]:
def plot_img(model, epoch):
    img = img_transformed.unsqueeze(0).to(DEVICE)
    # print(img.shape)
    pred = model(img)
    pred = t.sigmoid(pred)
    pred = t.where(pred <= 0.5, t.ones_like(pred, device=DEVICE), t.zeros_like(y_pred, device=DEVICE))
    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1)
    plt.imshow(img_transformed.permute(1, 2, 0).to('cpu'))
    plt.axis('off')
    plt.title('Original img')
    plt.subplot(1,3,2)
    plt.imshow(msk_transformed[0].to('cpu'), cmap='gray')
    plt.title('Original mask')
    plt.axis('off')
    plt.subplot(1,3,3)
    plt.imshow(pred[0][0].detach().to('cpu').numpy(), cmap='gray')
    plt.title('Predicted mask')
    plt.axis('off')
    plt.savefig(f"epoch_{epoch}.png")

In [4]:
def loss_iou(y_pred, y_true, inf):
    if not inf:
        if not y_pred.requires_grad:
            raise ValueError("y_pred should have gradient tracking")

    device = y_pred.device
    y_true = t.where(y_true <= 0, t.ones_like(y_pred, device=device), t.zeros_like(y_pred, device=device)).view((y_true.size(0), -1))
    y_pred = t.sigmoid(y_pred).view((y_true.size(0), -1))
    # y_pred = t.where(y_pred <= 0.5, t.zeros_like(y_pred, device=device), t.ones_like(y_pred, device=device)).requires_grad_(True)
    
    intersection = (y_pred * y_true).sum(dim=1) + 1e-5
    union = y_pred.sum(dim=1) + y_true.sum(dim=1) + 1e-5
    
    iou = intersection / (union - intersection)
    iou_loss = 1 - iou.mean()
    return iou_loss

In [ ]:
scaler = t.GradScaler(device=DEVICE)

for epoch in range(1, EPOCHS+1):
    model.train()
    train_loss_per_batch = 0
    bar = tqdm(range(len(train_dataloader)), desc="Batch processing", unit="batchs", colour='GREEN')
    for idx,batch in enumerate(train_dataloader):
        img = batch[0].float().to(DEVICE, non_blocking=True)
        mask = batch[1].float().to(DEVICE, non_blocking=True)

        # 1. Forward pass
        with t.autocast(device_type=DEVICE):
            y_pred = model(img)
            # 2. Calculate the loss
            ls = loss_iou(y_pred, mask, False)

        train_loss_per_batch += ls.item()

        optimizer.zero_grad(set_to_none=True)
        scaler.scale(ls).backward()
        scaler.step(optimizer=optimizer)
        scaler.update()
        bar.update(1)
        bar.set_postfix(loss = f"{ls.item():.4f}")
    bar.close()
    train_loss_per_batch /= idx+1

    model.eval()
    test_loss_per_batch = 0
    with t.inference_mode():
        bar = tqdm(range(len(val_dataloader)), desc="Batch processing", unit="batchs", colour='RED')
        for idx, batch in enumerate(val_dataloader):
            img = batch[0].float().to(DEVICE, non_blocking=True)
            mask = batch[1].float().to(DEVICE, non_blocking=True)

            y_pred_test = model(img)
            test_ls = loss_iou(y_pred_test, mask, True)

            test_loss_per_batch += test_ls.item()

            bar.update(1)
            bar.set_postfix(loss = f"{test_ls.item():.4f}")
        bar.close()

        test_loss_per_batch /= idx+1
        plot_img(model, epoch)

    print(f"{epoch} / {EPOCHS} | train_loss = {train_loss_per_batch:.4f} | test_loss = {test_loss_per_batch:.4f}")
    t.save(model.state_dict(), os.path.join(MODEL_SAVE_PATH, f"Image_enhancement_sd-{epoch}.pt"))
# t.save(model, "Image_enhancement-em.pt")

Batch processing: 100%|███████████████████████████████████████████████| 91/91 [50:04<00:00, 33.02s/batchs, loss=0.9316]


1 / 50 | train_loss = 0.9433 | test_loss = 0.9436


Batch processing:   1%|▏                                            | 2/365 [01:11<3:40:42, 36.48s/batchs, loss=0.9324]